In [1]:
import pandas as pd 

df = pd.read_csv("GermanCredit.csv")

#count 'none' values per column 
none_counts = df.apply(lambda col: col.value_counts().get('none', 0))
columns_to_drop = none_counts.nlargest(3).index 

#drop the columns 
df = df.drop(columns=columns_to_drop)
print("Dropped columns:", columns_to_drop.tolist())

Dropped columns: ['other_parties', 'other_payment_plans', 'own_telephone']
